In [ ]:
%tensorflow_version 2.x
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q efficientnet
import os
import re
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
!pip install gcsfs
from tqdm.notebook import tqdm as tqdm

!pip install tensorflow~=2.2.0 tensorflow_gcs_config~=2.2.0
import requests
resp = requests.post("http://{}:8475/requestversion/{}".format(os.environ["COLAB_TPU_ADDR"].split(":")[0], tf.__version__))
if resp.status_code != 200:
  print("Failed to switch the TPU to TF {}".format(version))

try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.15.92.98:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.15.92.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.15.92.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_PATH = 'gs://kds-4c1ac9004efd00b8a8bcfa9b3dcc800dbdf8978d612363a73d982469'
GCS_PATH_2 = 'gs://kds-e44a20ea6436cd0271b97c5b431450bcd70a463206a983e1acb88874'
DICT_PATH = 'gs://kds-c188e2ad51aa222c6ee46699b98bb511c8f660f45dcbb3344004e8be/train_encoded.csv'

# Configuration original batch is 32*num
EPOCHS = 30
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMAGE_SIZE = [384, 384]
# Seed
SEED = 100
# Learning rate
LR = 0.0001
# Number of classes
NUMBER_OF_CLASSES = 81313

# Training filenames directory
FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec') + tf.io.gfile.glob(GCS_PATH_2 + '/train*.tfrec')
# Read csv file
df = pd.read_csv(DICT_PATH)
# Using 2% of the data to validate
TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.001, random_state = SEED)
training_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in TRAINING_FILENAMES]
validation_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in VALIDATION_FILENAMES]
n_trn_classes = df[df['group'].isin(training_groups)]['landmark_id_encode'].nunique()
n_val_classes = df[df['group'].isin(validation_groups)]['landmark_id_encode'].nunique()
print(f'The number of unique training classes is {n_trn_classes} of {NUMBER_OF_CLASSES} total classes')
print(f'The number of unique validation classes is {n_val_classes} of {NUMBER_OF_CLASSES} total classes')

# Seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# Function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, (512, 512))
    # Convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    # Explicit size needed for TPU
    # image = tf.reshape(image, [*IMAGE_SIZE, 3])
    image = tf.reshape(image, [512, 512, 3])
    return image

# def read_image(image_path, size = (384, 384)):
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, size)
#     img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
#     img = tf.image.decode_jpeg(img, channels = 3)
#     img = tf.image.resize(img, (512, 512))
#     img = tf.cast(img, tf.float32) / 255.0
#     img = tf.reshape(img, [1, 512, 512, 3])
#     return img

# This function parse our images and also get the target variable
def read_tfrecord(example):
    TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64)
        }
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    target = tf.cast(example['target'], tf.int32)
    return image, target

# This function load our tf records and parse our data with the previous function
def load_dataset(filenames, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # Disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # Automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs
    dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# This function output the data so that we can use arcface
def arcface_format(image, target):
    return {'inp1': image, 'inp2': target}, target

# Training data pipeline
def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # The training dataset must repeat for several epochs
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Validation data pipeline
def get_validation_dataset(filenames, ordered = True, prediction = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    # If we are in prediction mode, use bigger batch size for faster prediction
    if prediction:
        dataset = dataset.batch(BATCH_SIZE * 4)
    else:
        dataset = dataset.batch(BATCH_SIZE)
    # Prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

# Count the number of observations with the tabular csv
def count_data_items(filenames):
    records = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    df = pd.read_csv(DICT_PATH)
    n = df[df['group'].isin(records)].shape[0]
    return n

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Training with {NUM_TRAINING_IMAGES} images')
print(f'Validating with {NUM_VALIDATION_IMAGES} images')

# Function for a custom learning rate scheduler with warmup and decay
#ramp_ep originally 5
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.0000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 8
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
    return lr_callback

# Function to calculate global average precision score
def gap_vector(pred, conf, true, return_x = False):
    '''
    Compute Global Average Precision (aka micro AP), the metric for the
    Google Landmark Recognition competition. 
    This function takes predictions, labels and confidence scores as vectors.
    In both predictions and ground-truth, use None/np.nan for "no label".

    Args:
        pred: vector of integer-coded predictions
        conf: vector of probability or confidence scores for pred
        true: vector of integer-coded labels for ground truth
        return_x: also return the data frame used in the calculation

    Returns:
        GAP score
    '''
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending = False, inplace = True, na_position = 'last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


# Function to build our model using fine tunning (efficientnet)
# original m = 0.05
def get_model():

    with strategy.scope():

        margin = ArcMarginProduct(
            n_classes = NUMBER_OF_CLASSES, 
            s = 64, 
            m = 0.15, 
            name='head/arc_margin', 
            dtype='float32'
            )

        # inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        inp = tf.keras.layers.Input(shape = (512, 512, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        # x4 = efn.EfficientNetB4(weights = 'imagenet', include_top = False)(inp)
        x6 = efn.EfficientNetB6(weights = None , include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x6)  
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(512)(x)
        x = margin([x, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)

        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)
        # opt = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=False, name='SGD', **kwargs)
        
        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model

# Seed everything
seed_everything(SEED)

# Build training and validation generators
train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
val_dataset = get_validation_dataset(VALIDATION_FILENAMES, ordered = True, prediction = False)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

model = get_model()
model.load_weights('/content/drive/My Drive/effb6512/effb6model512-20.h5')

# Using a checkpoint to save best model (want the entire model, not only the weights)
#include epoch names in each training step

checkpoint_path = '/content/drive/My Drive/effb6512/effb6model512-{epoch:02d}.h5'


checkpoint = tf.keras.callbacks.ModelCheckpoint( filepath = checkpoint_path, 
                                                 monitor = 'val_loss', 
                                                 save_best_only = False, 
                                                 save_weights_only = False,
                                                 period = 1)
# Using learning rate scheduler
cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', 
                                                       mode = 'min', 
                                                       factor = 0.5, 
                                                       patience = 1, 
                                                       verbose = 1, 
                                                       min_delta = 0.0001)


# Train and evaluate our model
history = model.fit(train_dataset,  
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = EPOCHS,
                    callbacks = [get_lr_callback(), checkpoint],
                    validation_data = val_dataset,
                    verbose = 1,
                    initial_epoch=20
                    )
#                    initial_epoch=23

The number of unique training classes is 81313 of 81313 total classes
The number of unique validation classes is 51634 of 81313 total classes
Training with 1475106 images
Validating with 105364 images


Epoch 19/30
23048/23048 [==============================] - 13285s 576ms/step - sparse_categorical_accuracy: 0.7848 - loss: 1.6652 - val_sparse_categorical_accuracy: 0.6561 - val_loss: 3.1983 - lr: 4.3286e-06
Epoch 20/30
23048/23048 [==============================] - 13338s 579ms/step - sparse_categorical_accuracy: 0.7909 - loss: 1.6195 - val_sparse_categorical_accuracy: 0.6599 - val_loss: 3.1616 - lr: 3.6629e-06
Epoch 21/30
 7447/23048 [========>.....................] - ETA: 2:28:28 - sparse_categorical_accuracy: 0.7852 - loss: 1.6467Buffered data was truncated after reaching the output size limit.

In [ ]:
model.save('/content/drive/My Drive/effb6512/effb6model512.h5')